# BatchBacktesting

In [1]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
import sys
import os
import httpx

In [3]:
from src.strategies import *
import concurrent.futures
from datetime import datetime
import glob
import warnings
from rich.progress import track
warnings.filterwarnings("ignore")

In [4]:
def make_api_request(api_endpoint, params):
    with httpx.Client() as client:
        # Make the GET request to the API
        response = client.get(api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        print("Error: Failed to retrieve data from API")
        return None


In [5]:
BASE_URL_FMP = "https://financialmodelingprep.com/api/v3"
BASE_URL_BINANCE = "https://fapi.binance.com/fapi/v1/"

FMP_API_KEY = "17c09553207c6d6e7bab10003e604aa8"
BINANCE_API_KEY = "17754325821w1b1e2khd34561hc54hh1"


def get_historical_price_full_crypto(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/crypto/{symbol}"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)


def get_historical_price_full_stock(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/{symbol}"
    params = {"apikey": FMP_API_KEY}

    return make_api_request(api_endpoint, params)

def get_SP500():
    api_endpoint = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    data = pd.read_html(api_endpoint)
    return list(data[0]['Symbol'])

def get_all_crypto():
    """
    All possible crypto on binance
    """

    api_endpoint = "https://api.binance.com/api/v3/ticker/price"
    data = make_api_request(api_endpoint, {})
    return [i['symbol'] for i in data]


def get_financial_statements_lists():
    api_endpoint = f"{BASE_URL_FMP}/financial-statement-symbol-lists"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)

In [6]:
def run_backtests_strategies(instruments, strategies):
    """
    Run backtests for a list of instruments using a specified strategy.

    Args:
        instruments (list): List of instruments to run backtests for
        strategies (list): List of strategies to run backtests for

    Returns:
        List of outputs from run_backtests()

    """

    # find strategies in the STRATEGIES
    strategies = [x for x in STRATEGIES if x.__name__ in strategies]
    outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for strategy in strategies:
            future = executor.submit(run_backtests, instruments, strategy, 4)
            futures.append(future)

        for future in concurrent.futures.as_completed(futures):
            outputs.extend(future.result())

    return outputs

def check_crypto(instrument):
    """
    Check if the instrument is crypto or not
    """
    return instrument in get_all_crypto()

def check_stock(instrument):
    """
    Check if the instrument is crypto or not
    """
    return instrument not in get_financial_statements_lists()


def process_instrument(instrument, strategy):
    """
    Process a single instrument for a backtest using a specified strategy.
    Returns a Pandas dataframe of the backtest results.
    """
    try:

        if check_crypto(instrument):
            data = get_historical_price_full_crypto(instrument)
        else:
            data = get_historical_price_full_stock(instrument)

        if data is None or "historical" not in data:
            print(f"Error processing {instrument}: No data")
            return None

        data = clean_data(data)

        bt = Backtest(
            data, strategy=strategy, cash=100000, commission=0.002, exclusive_orders=True
        )
        output = bt.run()
        output = process_output(output, instrument, strategy)
        return output, bt
    except Exception as e:
        print(f"Error processing {instrument}: {str(e)}")
        return None


def process_instrument_optimise(instrument, strategy):
    """
    Process a single instrument for a backtest using a specified strategy.
    Returns a Pandas dataframe of the backtest results.
    """
    try:
        data = get_historical_price_full_stock(instrument)
        data = clean_data(data)
        bt = Backtest(
            data, strategy=strategy, cash=100000, commission=0.002, exclusive_orders=True
        )
        output = bt.optimize_func()
        output = process_output(output, instrument, strategy)
        return output, bt
    except Exception as e:
        print(f"Error processing {instrument}: {str(e)}")
        return None


def clean_data(data):
    """
    Clean historical price data for use in a backtest.
    Returns a Pandas dataframe of the cleaned data.
    """
    data = data["historical"]
    data = pd.DataFrame(data)
    data.columns = [x.title() for x in data.columns]
    data = data.drop(
        [
            "Adjclose",
            "Unadjustedvolume",
            "Change",
            "Changepercent",
            "Vwap",
            "Label",
            "Changeovertime",
        ],
        axis=1,
    )
    data["Date"] = pd.to_datetime(data["Date"])
    data.set_index("Date", inplace=True)
    data = data.iloc[::-1]
    return data


def process_output(output, instrument, strategy, in_row=True):
    """
    Process backtest output data to include instrument name, strategy name,
    and parameters.
    Returns a Pandas dataframe of the processed output.
    """
    if in_row:
        output = pd.DataFrame(output).T
    output["Instrument"] = instrument
    output["Strategy"] = strategy.__name__
    output.pop("_strategy")
    output["StrategyParameters"] = strategy.__dict__
    return output


def save_output(output, output_dir, instrument, start, end):
    """
    Save backtest output to file and generate chart if specified.
    """
    print(f"Saving output for {instrument}")
    fileNameOutput = f"{output_dir}/{instrument}-{start}-{end}.csv"
    output.to_csv(fileNameOutput)


def plot_results(bt, output_dir, instrument, start, end):
    print(f"Saving chart for {instrument}")
    fileNameChart = f"{output_dir}/{instrument}-{start}-{end}.html"
    bt.plot(filename=fileNameChart, open_browser=False)

def run_backtests(instruments, strategy, num_threads=4, generate_plots=False):
    """
    Run backtests for a list of instruments using a specified strategy.
    Returns a list of Pandas dataframes of the backtest results.

    Args:
        instruments (list): List of instruments to run backtests for

    Returns:
        List of Pandas dataframes of the backtest results
    """
    outputs = []
    output_dir = f"output/raw/{strategy.__name__}"
    output_dir_charts = f"output/charts/{strategy.__name__}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(output_dir_charts):
        os.makedirs(output_dir_charts)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_instrument = {
            executor.submit(process_instrument, instrument, strategy): instrument
            for instrument in instruments
        }
        for future in concurrent.futures.as_completed(future_to_instrument):
            instrument = future_to_instrument[future]
            output = future.result()
            if output is not None:
                outputs.append(output[0])
                save_output(output[0], output_dir, instrument, output[0]["Start"].to_string().strip().split()[1], output[0]["End"].to_string().strip().split()[1])
                if generate_plots:
                    plot_results(output[1], output_dir_charts, instrument, output[0]["Start"].to_string().strip().split()[1], output[0]["End"].to_string().strip().split()[1])
    data_frame = pd.concat(outputs)
    start = data_frame["Start"].to_string().strip().split()[1]
    end = data_frame["End"].to_string().strip().split()[1]
    fileNameOutput = f"output/{strategy.__name__}-{start}-{end}.csv"
    data_frame.to_csv(fileNameOutput)


    return data_frame


def run_backtests_optimise(instruments, strategy, num_threads=4, generate_plots=False):
    """
    Run backtests for a list of instruments using a specified strategy.
    Returns a list of Pandas dataframes of the backtest results.
    """
    outputs = []
    output_dir = f"output/raw/{strategy.__name__}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_instrument = {
            executor.submit(process_instrument_optimise, instrument, strategy): instrument
            for instrument in track(instruments)
        }
        for future in track(concurrent.futures.as_completed(future_to_instrument)):
            instrument = future_to_instrument[future]
            output = future.result()
            if output is not None:
                outputs.append(output[0])
                save_output(output[0], output_dir, instrument, output[0]["Start"].to_string().strip().split()[1], output[0]["End"].to_string().strip().split()[1])
    data_frame = pd.concat(outputs)
    start = data_frame["Start"].to_string().strip().split()[1]
    end = data_frame["End"].to_string().strip().split()[1]
    fileNameOutput = f"output/{strategy.__name__}-{start}-{end}.csv"
    if generate_plots:
        plot_results(output[1], output_dir, instrument, start, end)

    return data_frame

In [7]:
tickers = get_SP500()
run_backtests(tickers, strategy=EMA, num_threads=8, generate_plots=True)

Saving output for AOS
Saving chart for AOS
Saving output for ADM
Saving chart for ADM
Saving output for MMM
Saving chart for MMM
Saving output for ABT
Saving chart for ABT
Saving output for ATVI
Saving chart for ATVI
Saving output for ABBV
Saving chart for ABBV
Saving output for ACN
Saving chart for ACN
Saving output for ADBE
Saving chart for ADBE
Saving output for ADP
Saving chart for ADP
Saving output for AAP
Saving chart for AAP
Saving output for AES
Saving chart for AES
Saving output for APD
Saving chart for APD
Saving output for A
Saving chart for A
Saving output for ALK
Saving chart for ALK
Saving output for AFL
Saving chart for AFL
Saving output for AKAM
Saving chart for AKAM
Saving output for ALB
Saving chart for ALB
Saving output for ARE
Saving chart for ARE
Saving output for ALGN
Saving chart for ALGN
Saving output for ALLE
Saving chart for ALLE
Saving output for LNT
Saving chart for LNT
Saving output for GOOGL
Saving chart for GOOGL
Saving output for GOOG
Saving chart for GO

In [ ]:
ticker = get_all_crypto()
run_backtests(ticker, strategy=EMA, num_threads=8, generate_plots=True)

Error processing NEOBTC: 'historical'
Error processing ETHBTC: 'historical'
Error processing LTCBTC: 'historical'
Error processing BNBBTC: 'historical'
Error processing EOSETH: 'historical'
Error processing BNTETH: 'historical'
Error processing QTUMETH: 'historical'
Error processing SNTETH: 'historical'
Error processing BCCBTC: 'historical'
Error processing BNBETH: 'historical'
Error processing GASBTC: 'historical'
Error processing BTCUSDT: 'historical'
Error processing ETHUSDT: 'historical'
Error processing HSRBTC: 'historical'
Error processing DNTETH: 'historical'
Error processing OAXETH: 'historical'
Error processing ICNETH: 'historical'Error processing MCOETH: 'historical'

Error processing WTCBTC: 'historical'
Error processing WTCETH: 'historical'
Error processing MCOBTC: 'historical'
Error processing LRCBTC: 'historical'
Error processing LRCETH: 'historical'
Error processing QTUMBTC: 'historical'
Error processing OMGBTC: 'historical'
Error processing YOYOBTC: 'historical'
Error p

In [ ]:
# run_backtests(get_financial_statements_lists(), EMA, num_threads=4)

In [ ]:
tickers = get_SP500()
run_backtests(tickers, strategy=RSI, num_threads=8, generate_plots=True)

In [ ]:
ticker = get_all_crypto()
run_backtests(ticker, strategy=RSI, num_threads=8, generate_plots=True)

In [ ]:
# run_backtests(get_financial_statements_lists(), RSI, num_threads=4)

In [ ]:
tickers = get_SP500()
run_backtests_strategies(tickers, ["LinearRegression"], generate_plots=True)

In [ ]:
tickers = get_all_crypto()
run_backtests_strategies(tickers, ["LinearRegression"], generate_plots=True)

In [ ]:
tickers = get_SP500()
#run_backtests(tickers, strategy=AroonOscillator, num_threads=4, generate_plots=True) # not working

In [ ]:
tickers = get_all_crypto()
#run_backtests(tickers, strategy=AroonOscillator, num_threads=4, generate_plots=True) # not working

In [ ]:
tickers = get_SP500()
run_backtests_strategies(tickers, STRATEGIES_STR, generate_plots=True)

In [ ]:
tickers = get_all_crypto()
run_backtests_strategies(tickers, STRATEGIES_STR, generate_plots=True)

NameError: name 'get_all_crypto' is not defined